In [1]:
#This notebook will contain calculations and analysis for the travel time of an average subway rider vs. an handicapped rider

In [2]:
#First I have two lists of random NYC addresses
import pandas as pd
import numpy as np

A_addresses = pd.read_csv('NYC_addresses_1.csv')
B_addresses = pd.read_csv('NYC_addresses_2.csv')

In [4]:
A_addresses.head()

,address_full,number,street,city,state,zip,longitude,latitude,statefp,countyfp,tractce,blkgrpce
0,"133-20 85 ST NEW YORK, NY 11417",133-20,85 ST,NEW YORK,NY,11417,-73.851338,40.673495,36,81,5800,3
1,"292 MONTAUK AVE NEW YORK, NY 11208",292,MONTAUK AVE,NEW YORK,NY,11208,-73.877478,40.671479,36,47,119400,3
2,"6 ROMER RD NEW YORK, NY 10304",6,ROMER RD,NEW YORK,NY,10304,-74.103098,40.593125,36,85,17701,3
3,"164-10 104 ST NEW YORK, NY 11414",164-10,104 ST,NEW YORK,NY,11414,-73.827662,40.651995,36,81,88400,6
4,"1429 SHORE PKWY NEW YORK, NY 11214",1429,SHORE PKWY,NEW YORK,NY,11214,-74.005699,40.598670,36,47,28200,1


In [5]:
B_addresses.head()

,address_full,number,street,city,state,zip,longitude,latitude,statefp,countyfp,tractce,blkgrpce
0,"3236 BOSTON RD NEW YORK, NY 10469",3236,BOSTON RD,NEW YORK,NY,10469,-73.854792,40.872162,36,5,34800,1
1,"970 AMSTERDAM AVE NEW YORK, NY 10025",970,AMSTERDAM AVE,NEW YORK,NY,10025,-73.965107,40.801581,36,61,19500,4
2,"1623 DECATUR ST NEW YORK, NY 11385",1623,DECATUR ST,NEW YORK,NY,11385,-73.901360,40.694898,36,81,55900,1
3,"12-06 151 PL NEW YORK, NY 11357",12-06,151 PL,NEW YORK,NY,11357,-73.811475,40.791270,36,81,98700,2
4,"135 BCH 56 PL NEW YORK, NY 11692",135,BCH 56 PL,NEW YORK,NY,11692,-73.787031,40.590884,36,81,97202,1


In [73]:
origins = A_addresses['address_full'].tolist()
origins

['133-20 85 ST NEW YORK, NY 11417 ',
 '292 MONTAUK AVE NEW YORK, NY 11208 ',
 '6 ROMER RD NEW YORK, NY 10304 ',
 '164-10 104 ST NEW YORK, NY 11414 ',
 '1429 SHORE PKWY NEW YORK, NY 11214 ',
 '196 22 ST NEW YORK, NY 11232 ',
 '715 SAGAMORE ST NEW YORK, NY 10462 ',
 '4730 RICHARDSON AVE NEW YORK, NY 10470 ',
 '94-17 84 ST NEW YORK, NY 11416 ',
 '2732 MILL AVE NEW YORK, NY 11234 ']

In [74]:
destinations = B_addresses['address_full'].tolist()
destinations

['3236 BOSTON RD NEW YORK, NY 10469 ',
 '970 AMSTERDAM AVE NEW YORK, NY 10025 ',
 '1623 DECATUR ST NEW YORK, NY 11385 ',
 '12-06 151 PL NEW YORK, NY 11357 ',
 '135 BCH 56 PL NEW YORK, NY 11692 ',
 '196 ROSEDALE AVE NEW YORK, NY 10312 ',
 '519 MADISON AVE NEW YORK, NY 10022 ',
 '197-26 CARPENTER AVE NEW YORK, NY 11423 ',
 '63-62 77 ST NEW YORK, NY 11379 ',
 '120-36 143 ST NEW YORK, NY 11436 ']

In [11]:
#That was the easy part. Now, gotta build an automated browser to run imput these addresses. 
import os
import random
import time

from playwright.async_api import async_playwright, expect, Keyboard

In [12]:
os.makedirs('data/', exist_ok=True)

In [13]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36'

In [66]:
async def open_browser(headless=False, user_agent=user_agent):
    """
    Starts the automated browser and opens a new window
    """
    # Start playwright
    playwright = await async_playwright().start()

    # Open firefox browser, can use chromium (chrome) or others
    browser = await playwright.chromium.launch(headless=False)

    # set a user agent
    context = await browser.new_context(user_agent=user_agent)
  
    # Create a new browser window
    page = await browser.new_page()

    return browser, page

In [67]:
driver, page = await open_browser()

In [68]:
url = 'https://www.mta.info/'
await page.goto(url)

<Response url='https://www.mta.info/' request=<Request url='https://www.mta.info/' method='GET'>>

In [69]:
from_search = page.locator('//*[@id="edit-origin"]')
from_search

<Locator frame=<Frame name= url='https://www.mta.info/'> selector='//*[@id="edit-origin"]'>

In [70]:
await from_search.fill('853 Riverside Dr, New York, NY, USA')

In [71]:
await page.mouse.click(200,450)

In [65]:
to_search = page.locator('//*[@id="edit-dest"]')
to_search

<Locator frame=<Frame name= url='https://www.mta.info/'> selector='//*[@id="edit-dest"]'>

In [55]:
await to_search.fill('75 9th Ave, New York, NY, USA')

In [56]:
await page.mouse.click(0,5)

In [57]:
submit = page.locator('//*[@id="edit-submit"]')
submit

<Locator frame=<Frame name= url='https://www.mta.info/'> selector='//*[@id="edit-submit"]'>

In [58]:
await submit.click()

In [94]:
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('api_key')

In [109]:
import requests

curl -X POST -d '{
  "origin":{
    "address": "853 Riverside Drive, New York, NY"
      },
  "destination":{
    "address": "75 9th Avenue, New York, NY" 
      },
  "travelMode": "TRANSIT",
  "computeAlternativeRoutes": true,
  "transitPreferences": {
    "allowedTravelModes": ["SUBWAY","BUS"]
  },
  "languageCode": "en-US",
  "units": "METRIC"
 }' \
-H 'Content-Type: application/json' 
-H 'X-Goog-Api-Key: api_key' \
-H 'X-Goog-FieldMask: routes.duration,routes.distanceMeters,routes.transitDetails' \
'https://routes.googleapis.com/directions/v2:computeRoutes'

SyntaxError: unterminated string literal (detected at line 3) (1632780211.py, line 3)

In [144]:
import requests

headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': api_key,
    'X-Goog-FieldMask': 'routes.legs.steps.transitDetails,routes.localizedValues'}

json_data = {
    'origin': {
        'address': '853 Riverside Drive, New York, NY',
    },
    'destination': {
        'address': '75 9th Avenue, New York, NY',
    },
    'travelMode': 'TRANSIT',
    'computeAlternativeRoutes': True,
    'transitPreferences': {
        'allowedTravelModes': [
            'SUBWAY',
            'BUS',
        ],
    },
    'languageCode': 'en-US',
    'units': 'METRIC',
}

response = requests.post('https://routes.googleapis.com/directions/v2:computeRoutes', headers=headers, json=json_data)

# Note: json_data will not be serialized by requests
# exactly as it was in the original request.
#data = '{\n  "origin":{\n    "address": "853 Riverside Drive, New York, NY"\n      },\n  "destination":{\n    "address": "75 9th Avenue, New York, NY" \n      },\n  "travelMode": "TRANSIT",\n  "computeAlternativeRoutes": true,\n  "transitPreferences": {\n    "allowedTravelModes": ["SUBWAY","BUS"]\n  },\n  "languageCode": "en-US",\n  "units": "METRIC"\n }'
#response = requests.post('https://routes.googleapis.com/directions/v2:computeRoutes', headers=headers, data=data)

In [145]:
data =response.json()

In [146]:
data

{'routes': [{'legs': [{'steps': [{},
      {},
      {},
      {},
      {},
      {'transitDetails': {'stopDetails': {'arrivalStop': {'name': '168 St',
          'location': {'latLng': {'latitude': 40.840557,
            'longitude': -73.94013199999999}}},
         'arrivalTime': '2025-07-29T12:53:24Z',
         'departureStop': {'name': '157 St',
          'location': {'latLng': {'latitude': 40.834042,
            'longitude': -73.944893}}},
         'departureTime': '2025-07-29T12:51:24Z'},
        'localizedValues': {'arrivalTime': {'time': {'text': '8:53\u202fAM'},
          'timeZone': 'America/New_York'},
         'departureTime': {'time': {'text': '8:51\u202fAM'},
          'timeZone': 'America/New_York'}},
        'headsign': 'Van Cortlandt Park-242 St',
        'headway': '300s',
        'transitLine': {'agencies': [{'name': 'MTA New York City Transit',
           'phoneNumber': '+1 718-330-1234',
           'uri': 'http://www.mta.info/'}],
         'name': '1 Train (Broadway

In [148]:
data['routes'][0]

{'legs': [{'steps': [{},
    {},
    {},
    {},
    {},
    {'transitDetails': {'stopDetails': {'arrivalStop': {'name': '168 St',
        'location': {'latLng': {'latitude': 40.840557,
          'longitude': -73.94013199999999}}},
       'arrivalTime': '2025-07-29T12:53:24Z',
       'departureStop': {'name': '157 St',
        'location': {'latLng': {'latitude': 40.834042,
          'longitude': -73.944893}}},
       'departureTime': '2025-07-29T12:51:24Z'},
      'localizedValues': {'arrivalTime': {'time': {'text': '8:53\u202fAM'},
        'timeZone': 'America/New_York'},
       'departureTime': {'time': {'text': '8:51\u202fAM'},
        'timeZone': 'America/New_York'}},
      'headsign': 'Van Cortlandt Park-242 St',
      'headway': '300s',
      'transitLine': {'agencies': [{'name': 'MTA New York City Transit',
         'phoneNumber': '+1 718-330-1234',
         'uri': 'http://www.mta.info/'}],
       'name': '1 Train (Broadway-7 Av Local)',
       'uri': 'https://new.mta.info/docum